In [9]:
import sys
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Different Paths
alg_path = "/workspaces/CP1-24-final/HeshamElsaman/Code/Algorithm"
data_path = "/workspaces/CP1-24-final/HeshamElsaman/Data/GPSSine"
sys.path.append(alg_path)
sys.path.append(data_path)

import final

In [10]:
# Reloading the final module in case of changes taking place
importlib.reload(final)

<module 'final' from '/workspaces/CP1-24-final/HeshamElsaman/Code/Algorithm/final.py'>